In [1]:
from pynq import Overlay
overlay = Overlay('/home/xilinx/jupyter_notebooks/testFPGA/top_graph.bit')

In [10]:
from pynq import allocate
import numpy as np
import time

N = 64

# Buffer creation
in1_buffer = allocate(shape=(N,), dtype=np.int32)
in2_buffer = allocate(shape=(N,), dtype=np.int32)
add_buffer = allocate(shape=(N,), dtype=np.int32)
mul_buffer = allocate(shape=(N,), dtype=np.int32)

# TODO fill data
in1 = [i for i in range(N)]
in2 = [i * i for i in range(N)]

# data copy to buffer
np.copyto(in1_buffer, in1)
np.copyto(in2_buffer, in2)

# Buffer mapping to FPGA
mem_read = overlay.mem_read_0
mem_read.write(mem_read.register_map.in1_mem_1.address, in1_buffer.physical_address)
mem_read.write(mem_read.register_map.in2_mem_1.address, in2_buffer.physical_address)
mem_write = overlay.mem_write_0
mem_write.write(mem_write.register_map.outAdd_mem_1.address, add_buffer.physical_address)
mem_write.write(mem_write.register_map.outMul_mem_1.address, mul_buffer.physical_address)

# Start read and write
start = time.time()
mem_read.register_map.CTRL.AP_START = 1
mem_write.register_map.CTRL.AP_START = 1
end = time.time()

# Active wait
while mem_write.register_map.CTRL.AP_DONE == 0:
    end = time.time()
print ("time:", end - start)

# TODO check results
correct = True
for i in range(N):
    correct = correct and in1[i] + in2[i] == add_buffer[i]
    correct = correct and in1[i] * in2[i] == mul_buffer[i]
    
print(correct)

time: 0.001730203628540039
True
